# Detección de tweets meteorológicos con Bag of Words

In [77]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

#### Cargo la base de datos clasificada y genero las variables de importancia

In [78]:
tw = pd.read_csv('base_corr.csv')

In [79]:
tw.keys()[-3:]

Index(['Twit', 'Tag', 'Num'], dtype='object')

In [89]:
msj = tw['Twit']
tag = tw['Tag']
num = tw['Num']

In [90]:
msj

0                                         Esta lloviendo
1      quiero ir al centro a la mañana pero seguro ll...
2                   @paloma_CAI7 Si, llueve y mucho !! 😒
3                       @rozalazar9 Si llueve les hago 🤗
4      Hasta cuándo le voy a seguir teniendo miedo a ...
5      13h ClimaRafaela: Temp. 20.9 °C, Hum. 48 %, Ba...
6      @Eze2450 Sea la estación que sea , no hay plat...
7      Llueve y no tengo Internet...me espera tremend...
8                            Que linda noche de lluvia 💖
9      Miércoles de mucho calor, previo a las lluvias...
10     Me gusta cuando llueve porque parece que se es...
11     Noviembre sin ti es sentir que la lluvia me di...
12     17h Agvnet CDU: Temp. 32.4 °C, Hum. 33 %, Bar....
13      Con esta lluvia me dieron ganas de cucharear ! 😏
14                               Y se largo la lluvia ☔💕
15                                  Que hermosa tormenta
16                         @joaquinv17 dan lluvia jajaja
17     Aseguran que hay stock s

#### Elimino las cosas innecesarias de msj

In [290]:
def normalizacion(mensaje):
    
    mensaje = list(mensaje)
    lenght = len(mensaje)
    
    lista_ascii = [['\\xe1', '\\xe9', '\\xed', '\\xf3', '\\xfa', '\\xf1', '\\xd1', '\\xc1','\\xc9', '\\xcd', '\\xd3', 
                 '\\xda', '\\xa1', '\\xbf', '\\xdc', '\\xfc', '\\xb4', '\\xba', '\\xaa', '\\xb7','\\xc7','\\xe7',
                 '\\xa8', '\\xb0C', '\\n', '\\xb0c', '\\xbb', 'xb0'],['á', 'é', 'í', 'ó', 'ú', 'ñ', 'Ñ', 'Á', 'É', 'Í', 'Ó', 'Ú', '', '', 'Ü', 'ü', '', 
                           ' ', '', '','Ç','ç','', ' ', ' ', ' ', ' ', ' ']]
    lista_simb = ['!','\'', '\"', "|", '$', "%", "&", "(", ")", "=", "?", "+",'/', ";", "_", "-", "1","2","3","4","5","6","7","8","9","0", '*',',', '.', ':', '#']
    
    for i in range(lenght):
        
        ## Convierto mayúsculas en minúsculas
        mensaje[i] = mensaje[i].lower()
        
        ## Saco las menciones y otras cosas
        txt = mensaje[i].split()
        
        for j in range(len(txt)):
            if ('@' in txt[j]) and ('RELAMPAGO2018' not in txt[j]) and ('RELAMPAGO_edu' not in txt[j]) or ('jaj' in txt[j]) or ('https' in txt[j]): txt[j]=''
        
        mensaje[i] = " ".join(txt)
        
        ## Reemplazo símbolos
        for h in range(len(lista_simb)):
            mensaje[i] = mensaje[i].replace(lista_simb[h], ' ')
            
        ## Convierto el msj a ASCII, reemplazo los errores de decodificación y agrego un espacio antes de cada decodificación
        mensaje[i] = mensaje[i].encode('unicode-escape').decode('ASCII')+" "     
        
        for j in range(len(lista_ascii[0])):
            mensaje[i] = mensaje[i].replace(lista_ascii[0][j], lista_ascii[1][j])
        
        mensaje[i] = mensaje[i].replace('\\', ' \\')
        
    return(mensaje)

In [291]:
msj_corr = normalizacion(msj)

In [292]:
msj_corr

['esta lloviendo ',
 'quiero ir al centro a la mañana pero seguro llueve  ',
 ' si  llueve y mucho  \\U0001f612 ',
 ' si llueve les hago  \\U0001f917 ',
 'hasta cuándo le voy a seguir teniendo miedo a las tormentas  \\U0001f644 \\U0001f623 ',
 '  h climarafaela  temp          hum        bar         mbar  lluvia     mm  viento      km h sso  weathercloud ',
 ' sea la estación que sea   no hay plata y ahora llueve   que te cambia  tenés que estudiar igual ',
 'llueve y no tengo internet   me espera tremenda tarde ',
 'que linda noche de lluvia  \\U0001f496 ',
 'miércoles de mucho calor  previo a las lluvias  ',
 'me gusta cuando llueve porque parece que se estan fritando empanadas  puesgorda  \\U0001f602 ',
 'noviembre sin ti es sentir que la lluvia me dice llorando que todo acabó  \\U0001f3b5 ',
 '  h agvnet cdu  temp          hum        bar        hpa  lluvia   mm  viento      km h se  weathercloud ',
 'con esta lluvia me dieron ganas de cucharear  \\U0001f60f ',
 'y se largo la lluvia